## Normalización

Cuando se normalizan las features de un DataFrame con StandardScaler de scikit-learn, cada feature se transforma de forma independiente usando su propio factor de escala (desviación estándar) y su propio valor de desplazamiento (media).

### ¿Cómo funciona StandardScaler?

StandardScaler transforma cada feature $ x_i $ de la siguiente manera:

$$
x_i^{\text{scaled}} = \frac{x_i - \mu_i}{\sigma_i}
$$

Donde:

- $ \mu_i $ es la **media** de la feature $ i $, calculada como:

  $$
  \mu_i = \frac{1}{n} \sum_{j=1}^{n} x_{ij}
  $$

- $ \sigma_i $ es la **desviación estándar** de la feature $ i $, calculada como:

  $$
  \sigma_i = \sqrt{\frac{1}{n} \sum_{j=1}^{n} (x_{ij} - \mu_i)^2}
  $$

Aquí:

- $ x_{ij} $ es el valor de la feature $ i $ para el ejemplo $ j $
- $ n $ es el número total de muestras


Cada feature se escala (y desplaza) con sus propios valores de media y desviación estándar.
Esto garantiza que cada columna del DataFrame resultante tenga media 0 y desviación estándar 1, permitiendo comparaciones más justas entre features con escalas diferentes.

No se ajusta el modelo para que funcione con las features sin normalizar.
En su lugar, se normalizan también las features nuevas (de entrada) antes de usarlas con el modelo entrenado.

### ¿Por qué no se "des-normaliza" el modelo?
Cuando entrenás un modelo con features normalizadas (por ejemplo, usando `StandardScaler`), los parámetros internos del modelo se ajustan a esas escalas. Cambiar las escalas de las features después sin volver a entrenar el modelo afectaría directamente su desempeño y precisión.

### Flujo correcto:

1. Durante el entrenamiento

```python
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

scaler = StandardScaler()
model = LinearRegression()

# Encapsular todo en un pipeline
pipeline = make_pipeline(scaler, model)

# Entrenar
pipeline.fit(X_train, y_train)
```

2. Durante la predicción (en producción):

```python
# Las nuevas features sin normalizar
X_new = ...

# El pipeline ya sabe cómo escalar porque guarda los parámetros del scaler
y_pred = pipeline.predict(X_new)
```

**✅ El StandardScaler guarda internamente la media y desviación estándar, y se usa para transformar los datos nuevos de forma coherente.**

**¿Y si ya tenés el modelo entrenado pero querés hacerlo funcionar con features sin escalar?**  
Tendrías que reentrenar el modelo directamente con los datos sin escalar, o hacer ingeniería inversa para "desescalar" los coeficientes del modelo, lo cual es complicado y propenso a errores (y solo posible en modelos lineales simples).

### Conclusión
* Nunca "desnormalices" el modelo.

* Siempre normalizá las nuevas features igual que las del entrenamiento.

* Usá un `Pipeline` de `scikit-learn` para que esto sea automático y robusto.

## Cómo **guardar** y **cargar** un `Pipeline` con `StandardScaler` y un modelo, usando `joblib` (recomendado por `scikit-learn` para este tipo de tareas).

### ✅ Paso 1: Entrenar, crear y guardar el pipeline

```python
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import joblib

# Entrenamiento
scaler = StandardScaler()
model = LinearRegression()

# Crear pipeline
pipeline = make_pipeline(scaler, model)

# Ajustar con los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Guardar el pipeline en un archivo
joblib.dump(pipeline, 'modelo_entrenado.pkl')

```

### ✅ Paso 2: Cargar el pipeline entrenado y hacer predicciones

```python
# Cargar el pipeline entrenado desde el archivo
pipeline_cargado = joblib.load('modelo_entrenado.pkl')

# Nuevos datos sin escalar
X_nuevo = ...

# Predecir directamente (el pipeline aplica el StandardScaler internamente)
y_pred = pipeline_cargado.predict(X_nuevo)

```

### 🔐 ¿Por qué esto es útil?
* No necesitás preocuparte por guardar `mean_` y `scale_` del scaler.

* Todo está encapsulado: el escalado y el modelo.

* Es ideal para producción o integración con APIs.

## Aquí va un ejemplo completo y funcional con datos ficticios:

### 🧪 Ejemplo completo: `StandardScaler` + `LinearRegression` + Pipeline  
### 📦 Librerías necesarias

```python
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import joblib

```

### 🧷 1. Generar datos ficticios

```python
# Datos: 100 samples, 2 features
X = np.random.rand(100, 2) * 100  # Valores entre 0 y 100
y = 3 * X[:, 0] + 2 * X[:, 1] + np.random.randn(100) * 10  # y con algo de ruido

# Separar en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

```

### ⚙️ 2. Crear, entrenar y guardar el pipeline

```python
# Crear pipeline con StandardScaler y regresión lineal
pipeline = make_pipeline(StandardScaler(), LinearRegression())

# Entrenar el pipeline
pipeline.fit(X_train, y_train)

# Guardar el pipeline en un archivo
joblib.dump(pipeline, 'modelo_escalado.pkl')

```

### 🧠 3. Cargar y usar el modelo en producción

```python
# Cargar el pipeline entrenado
modelo_cargado = joblib.load('modelo_escalado.pkl')

# Simular nuevos datos crudos (sin escalar)
X_nuevos = np.array([[10, 20], [50, 60]])

# Predecir
predicciones = modelo_cargado.predict(X_nuevos)

print("Predicciones:", predicciones)

```

### ✅ Salida esperada
Vas a ver las predicciones que el modelo entrenado genera para los datos `[10, 20]` y `[50, 60]`, aplicando internamente el `StandardScaler` con los parámetros aprendidos durante el entrenamiento.

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import joblib

# Datos: 100 samples, 2 features
X = np.random.rand(100, 2) * 100  # Valores entre 0 y 100
y = 3 * X[:, 0] + 2 * X[:, 1] + np.random.randn(100) * 10  # y con algo de ruido

# Separar en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear pipeline con StandardScaler y regresión lineal
pipeline = make_pipeline(StandardScaler(), LinearRegression())

# Entrenar el pipeline
pipeline.fit(X_train, y_train)

# Guardar el pipeline en un archivo
joblib.dump(pipeline, 'modelo_escalado.pkl')

# Cargar el pipeline entrenado
modelo_cargado = joblib.load('modelo_escalado.pkl')

# Simular nuevos datos crudos (sin escalar)
X_nuevos = np.array([[10, 20], [50, 60]])

# Predecir
predicciones = modelo_cargado.predict(X_nuevos)

print("Predicciones:", predicciones)


Predicciones: [ 64.80315401 268.28548109]


# ¿Para qué sirve `joblib`?

`joblib` es una biblioteca de Python muy usada para:

---

## ✅ Guardar y cargar objetos grandes y complejos de forma eficiente

Especialmente útil cuando trabajás con:

- Modelos entrenados (`scikit-learn`, `XGBoost`, etc.)
- `Pipeline`s (escaladores + modelos)
- Objetos de NumPy grandes (como matrices o vectores)
- Listas, diccionarios, funciones, clases, etc.

---

## 📦 ¿Para qué se usa normalmente?

### 1. Guardar modelos entrenados

```python
import joblib

joblib.dump(modelo_entrenado, 'modelo.pkl')
```

### 2. Cargar modelos ya entrenados

```python
modelo = joblib.load('modelo.pkl')
```

### 🆚 ¿Por qué no pickle?
* joblib está optimizado para trabajar con arrays grandes de NumPy, lo cual es muy común en machine learning.

* Es más rápido y eficiente que pickle para objetos como modelos de scikit-learn.

🔧 Internamente, joblib usa compresión y almacenamiento binario eficiente.

### 🧠 ¿Cuándo usar joblib?

| Situación                                | ¿Usar `joblib`?                |
| ---------------------------------------- | ------------------------------ |
| Guardar modelo de ML                     | ✅ Sí                           |
| Guardar grandes arrays de NumPy          | ✅ Sí                           |
| Guardar objetos comunes (texto, números) | ❌ Mejor usar `pickle` o `json` |
| Reutilizar pipelines o transformadores   | ✅ Sí                           |
